In [69]:
import gurobipy as gp
from gurobipy import GRB
import json
import pandas as pd
import numpy as np

In [70]:
"Loading the Solar Production Data";
with open(r"C:\Users\Pulin\Desktop\Optimisation_PowerSystems_Assignment1\data\question_1a\DER_production.json", "r") as f:
    data = json.load(f)

df = pd.DataFrame(data)

# Take the first row's hourly_profile_ratio list and make it a numpy array
Solar_CF = np.array(df["hourly_profile_ratio"][0])

In [71]:
Solar_CF

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.05, 0.14, 0.21, 0.15, 0.12, 0.21,
       0.25, 0.85, 0.75, 0.55, 0.43, 0.23, 0.05, 0.25, 0.25, 0.  , 0.  ,
       0.  , 0.  ])

In [72]:
"Fullt Flexible Load (FFL) Data";
# Load JSON filen
with open(r"C:\Users\Pulin\Desktop\Optimisation_PowerSystems_Assignment1\data\question_1a\usage_preference.json", "r") as f:
    data = json.load(f)

    data


In [73]:
min_FFL = data[0]["load_preferences"][0]["min_total_energy_per_day_hour_equivalent"]
min_FFL

8

In [74]:
"Electrcity Markert Data";
with open(r"C:\Users\Pulin\Desktop\Optimisation_PowerSystems_Assignment1\data\question_1a\bus_params.json", "r") as f:
    data = json.load(f)

electricty_price = data[0]["energy_price_DKK_per_kWh"]

In [75]:
price_import = data[0]["import_tariff_DKK/kWh"]
price_export = data[0]["export_tariff_DKK/kWh"]
#------------------------------------------
penalty_excess_import = data[0]["penalty_excess_import_DKK/kWh"]
penalty_excess_export = data[0]["penalty_excess_export_DKK/kWh"]
#-----------------------------------------------------
max_import = data[0]["max_import_kW"]
max_export = data[0]["max_export_kW"]

In [76]:
max_export

500

In [77]:
"Ramping Up for Solar and FFL";
with open(r"C:\Users\Pulin\Desktop\Optimisation_PowerSystems_Assignment1\data\question_1a\appliance_params.json", "r") as f:
    data = json.load(f)

max_power_solar = data["DER"][0]["max_power_kW"]
#min_power = data["DER"][0]["min_power_ratio"]
max_ramp_up_rate_solar = data["DER"][0]["max_ramp_rate_up_ratio"]
max_ramp_down_rate_solar = data["DER"][0]["max_ramp_rate_down_ratio"]

In [78]:
max_load_FFL = data["load"][0]["max_load_kWh_per_hour"]
min_load_FFL = data["load"][0]["min_load_ratio"]

max_ramp_up_rate_FFL= data["load"][0]["max_ramp_rate_up_ratio"]
max_ramp_down_rate_FFL= data["load"][0]["max_ramp_rate_down_ratio"]

In [85]:
max_load_FFL

3.0

- all units are in kW or KWh

In [83]:
t = range(len(Solar_CF))

In [108]:
# Create the model
model = gp.Model("Prosumer_Optimization")

# Decision variables
x_imports = model.addVars(t, name="imported_power_kW", lb=0, ub=GRB.INFINITY)  # x_import[t]
x_exports = model.addVars(t, name="exported_power_kW", lb=0, ub=GRB.INFINITY) # x_exports[t]
x_FFL = model.addVars(t, name="FFL_consumption_kW", lb=0, ub=1)  # scaled 0-1 # x_FFL[t]

z_import_excess = model.addVars(t, name="excess_import_kW", lb=0, ub=GRB.INFINITY) # z_import_excess[t]
z_export_excess = model.addVars(t, name="excess_export_kW", lb=0, ub=GRB.INFINITY) # z_import_excess[t], z_export_excess[t]

# Hourly binary variables
y_import = model.addVars(t, vtype=GRB.BINARY, name="tariff_import_activation") # y_import[t]
y_export = model.addVars(t, vtype=GRB.BINARY, name="tariff_export_activation") # y_export[t]

# Constraints for each hour
for i in t:
    # FFL constraints
    model.addConstr(x_FFL[i] * max_load_FFL >= min_load_FFL, name=f"FFL_min_{i}")
    model.addConstr(x_FFL[i] * max_load_FFL <= max_load_FFL, name=f"FFL_max_{i}")

    # Energy balance
    model.addConstr(
        x_imports[i] - x_exports[i] + Solar_CF[i] * max_power_solar - x_FFL[i] * max_load_FFL == 0,
        name=f"energy_balance_{i}")


    # Excess import/export constraints
    model.addConstr(x_imports[i] - max_import == z_import_excess[i], name=f"excess_import_{i}") # Quantity of excess import
    model.addConstr(x_exports[i] - max_export == z_export_excess[i], name=f"excess_export_{i}") # Quantity of excess export

    # Tariff activation (big-M constraints)
    model.addConstr( max_import >= x_imports[i] * (1 - y_import[i]), name=f"import_limit_{i}")
    model.addConstr( max_export >= x_exports[i] * (1 - y_export[i]), name=f"export_limit_{i}")

    # Ramp constraints for FFL only (solar is fixed)
    if i > 0:
        model.addConstr(
            (x_FFL[i] * max_load_FFL) - (x_FFL[i-1] * max_load_FFL) <= max_ramp_up_rate_FFL * max_load_FFL,
            name=f"FFL_ramp_up_{i}")
        model.addConstr(
            (x_FFL[i-1] * max_load_FFL) - (x_FFL[i] * max_load_FFL) <= max_ramp_down_rate_FFL * max_load_FFL,
            name=f"FFL_ramp_down_{i}")


# Objective: minimize import cost + excess import penalty
model.setObjective(
    gp.quicksum(penalty_excess_import*z_import_excess[i] + price_import*x_imports[i] +
                 penalty_excess_export*z_export_excess[i] + price_export*x_exports[i] 
                for i in t), 
                sense=GRB.MINIMIZE)

model.optimize()


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 8845HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 166 rows, 168 columns and 308 nonzeros
Model fingerprint: 0xb377fce5
Model has 48 quadratic constraints
Variable types: 120 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [4e-01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 1e+03]
  QRHS range       [5e+02, 1e+03]
Presolve removed 166 rows and 166 columns
Presolve time: 0.04s
Presolved: 3 rows, 5 columns, 7 nonzeros
Presolved model has 2 SOS constraint(s)
Variable types: 3 continuous, 2 integer (2 binary)

Root relaxation: objective 1.409913e+05, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    

In [109]:
# Print objective value
print("Optimal objective value:", model.ObjVal)

# Print hourly results
for i in t:
    print(f"Hour {i}:")
    print(f"  Imported power = {x_imports[i].X:.2f} kW")
    print(f"  Exported power = {x_exports[i].X:.2f} kW")
    print(f"  FFL consumption = {x_FFL[i].X*max_load_FFL:.2f} kW")
    print(f"  Excess import = {z_import_excess[i].X:.2f} kW")
    print(f"  Excess export = {z_export_excess[i].X:.2f} kW")
    print(f"  Import tariff active = {y_import[i].X}")
    print(f"  Export tariff active = {y_export[i].X}")


Optimal objective value: 140991.288
Hour 0:
  Imported power = 1000.00 kW
  Exported power = 997.00 kW
  FFL consumption = 3.00 kW
  Excess import = 0.00 kW
  Excess export = 497.00 kW
  Import tariff active = 0.0
  Export tariff active = 1.0
Hour 1:
  Imported power = 1000.00 kW
  Exported power = 997.00 kW
  FFL consumption = 3.00 kW
  Excess import = 0.00 kW
  Excess export = 497.00 kW
  Import tariff active = 0.0
  Export tariff active = 1.0
Hour 2:
  Imported power = 1000.00 kW
  Exported power = 997.00 kW
  FFL consumption = 3.00 kW
  Excess import = 0.00 kW
  Excess export = 497.00 kW
  Import tariff active = 0.0
  Export tariff active = 1.0
Hour 3:
  Imported power = 1000.00 kW
  Exported power = 997.00 kW
  FFL consumption = 3.00 kW
  Excess import = 0.00 kW
  Excess export = 497.00 kW
  Import tariff active = 0.0
  Export tariff active = 1.0
Hour 4:
  Imported power = 1000.00 kW
  Exported power = 997.00 kW
  FFL consumption = 3.00 kW
  Excess import = 0.00 kW
  Excess export